In [1]:
from typing import List
import seaborn as sns
import matplotlib as plt

import pandas as pd
import numpy as np
from cobra.io import read_sbml_model
from cobra.io.sbml import Model
from mewpy.simulation import Environment, Simulator, get_simulator

from concurrent.futures import ProcessPoolExecutor

In [2]:
ec_bt = read_sbml_model('../models/ec/ec_Bacteroides_thetaiotaomicron_VPI_5482.xml')
ec_bu = read_sbml_model('../models/ec/ec_Bacteroides_uniformis_ATCC_8492.xml')
ec_ec = read_sbml_model('../models/ec/ec_Escherichia_coli_ED1a.xml')
ec_fn = read_sbml_model('../models/ec/ec_Fusobacterium_nucleatum_subsp_nucleatum_ATCC_25586.xml')
ec_ri = read_sbml_model('../models/ec/ec_Roseburia_intestinalis_L1_82.xml')
ec_sp = read_sbml_model('../models/ec/ec_Streptococcus_parasanguinis_ATCC_15912.xml')
ec_ss = read_sbml_model('../models/ec/ec_Streptococcus_salivarius_DSM_20560.xml')
ec_cc = read_sbml_model('../models/ec/ec_Coprococcus_comes_ATCC_27758.xml')

sim_bt = get_simulator(ec_bt)
sim_bu = get_simulator(ec_bu)
sim_ec = get_simulator(ec_ec)
sim_cc = get_simulator(ec_cc)
sim_fn = get_simulator(ec_fn)
sim_ri = get_simulator(ec_ri)
sim_sp = get_simulator(ec_sp)
sim_ss = get_simulator(ec_ss)

sim_list = [sim_bt, sim_bu, sim_cc, sim_fn, sim_sp, sim_ss]

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-11


In [3]:
def prot_pool_analysis(sim: Simulator):
    l_va = np.linspace(0.1, 1.0, num=10)
    constraint_list: List = []
    sol_list: List = []
    ind_list: List = []

    for i in range(len(l_va)):
        constraint_list.append({'prot_pool_exchange': (0, round(l_va[i], 1))})
        ind_list.append(f'0 - {round(l_va[i], 1)}')

    model_id = str(sim.id.strip('M_'))
    print(f'Running {model_id}')
    #env = Environment.complete(sim, max_uptake=1000.0, inplace=False)
    #sim.set_environmental_conditions(env)
    for i in range(len(l_va)):
        cons = constraint_list[i]
        solution = sim.simulate(constraints=cons)
        sol_list.append(solution.objective_value)
    data = {'Protein pool exchange': ind_list, model_id: sol_list}
    df = pd.DataFrame(data)
    df = df.set_index('Protein pool exchange')
    return df

In [4]:
merged: List = []

with ProcessPoolExecutor() as p:
        solution_list = []
        for result in p.map(prot_pool_analysis, sim_list):
            merged.append(result)
        df = pd.concat(merged, axis=1)
        print(df)
        df_norm = (df - df.mean()) / df.std()
        sns.heatmap(df_norm)

Read LP format model from file /tmp/tmpupb7uhd9.lp
Reading time = 0.03 seconds
: 2865 rows, 9480 columns, 29512 nonzeros
Read LP format model from file /tmp/tmp80aepj4x.lp
Reading time = 0.03 seconds
: 3026 rows, 11046 columns, 37516 nonzeros
Read LP format model from file /tmp/tmp_w2jppg0.lp
Reading time = 0.02 seconds
: 1775 rows, 4488 columns, 18164 nonzeros
Read LP format model from file /tmp/tmp5lllk2r1.lp
Reading time = 0.02 seconds
: 1639 rows, 4620 columns, 17262 nonzeros
Read LP format model from file /tmp/tmps2vn96y0.lp
Reading time = 0.01 seconds
: 1378 rows, 3684 columns, 15016 nonzeros
Read LP format model from file /tmp/tmpm7ewhheg.lp
Reading time = 0.02 seconds
: 2131 rows, 5528 columns, 23710 nonzeros
Running Streptococcus_salivarius_DSM_20560
Running Coprococcus_comes_ATCC_27758
Running Fusobacterium_nucleatum_subsp_nucleatum_ATCC_25586
Running Streptococcus_parasanguinis_ATCC_15912
Running Bacteroides_thetaiotaomicron_VPI_5482
Running Bacteroides_uniformis_ATCC_8492


OptimizationError: Solver status is 'unbounded'.